In [ ]:
""" 
DEVELOPing stuff, 
mult day analysis, based on extracting experiments

3/14/21 - All moved to analysis/dataset.py. Code below should work in calling those.

[OBSOLETE] Extraction is from calling dataset.py as main.
"""


In [1]:
%load_ext autoreload
%autoreload 2

%cd ..

from tools.utils import * 
from tools.plots import *
from tools.analy import *
from tools.calc import *
from tools.analyplot import *
from tools.preprocess import *
from tools.dayanalysis import *
from analysis.line2 import *
from analysis.modelexpt import *
from analysis.probedatTaskmodel import *
from analysis.dataset import Probedat2Dat

from pythonlib.drawmodel.analysis import *
from pythonlib.tools.stroketools import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/data1/code/python/drawmonkey
NOTE: need to not overwrite strokes_all_task, because then the orders saved will stop being accurate. Modify


# MAIN CODE - generate and save datasetsm

## check for a given set of tasks

In [3]:
# exptlist = ["arc2", "lines5"]
SKIPS = []
DOSKIP = False
DO_NOT_LET_SKIP= True
# exptlist = ["biasdir2", "biasdir3", "biasdir4", "biasdir5", 
#             "biasdir6", "biasdir7_1", "biasdir7_2", "biasdir8", "biasdir9"]
# exptlist = ["neuralprep8", "neuralprep5", "neuralprep1", "neuralprep2", "neuralprep3", "neuralprep4",
#             "neuralprep6", "neuralprep7"]
exptlist = ["primitives2"]
animallist = ["Pancho"]
rulelist = ["null"]
# rulelist = [None]
# exptlist = ["neuralprep5"]
for expt in exptlist: 
    for animal in animallist:
        for rule in rulelist:
            try:
                FD, exptMetaDat = loadMultDataForExpt(expt,animal, metadatonly=False, rule=rule)
                PD = loadProbeDatWrapper(FD, exptMetaDat)
                P = ProbedatTaskmodel(PD, exptMetaDat)

                extraction_params = {
                    "expt":expt,
                    "animal":animal,
    #                 "probedat_filter_params":{
    #                     "hausdorff_filter":True,
    #                     "hausdorff_filter_prctile":2.5,
    #                     },
                    "probedat_filter_params":{},
                    "pix_add_to_sketchpad_edges":20,
                    "savedir":"/data2/analyses/database/BEH",
                    "savenote":"formodeling"
                }

                # ==== filter trials based on behavioral criteria, to throw out noise.
                ProbedatFiltered = P.filterByBehPerformance(extraction_params["probedat_filter_params"])

                # Reconstruct P
                P = ProbedatTaskmodel(ProbedatFiltered, P.Metadat)

                # === Convert Probedat to DAT
                DAT, METADAT = Probedat2Dat(P, extraction_params, save=True, keep_all_in_probedat=True)
            except:
                if DO_NOT_LET_SKIP:
                    raise
                if DOSKIP:
                    SKIPS.append([expt, animal])
                else:
                    raise



This is metadat:
{'sdate': 210606, 'edate': 210608, 'strokmodel_kind': None, 'strokmodel_tstamp': None, 'datecategories': {'210606': 1, '210607': 1, '210608': 1}, 'dates_for_summary': [], 'matchedstrokes': None, 'exptnames': ['primitives2'], 'T1': [], 'G1': [], 'G2': [], 'G3': [], 'G4': [], 'description': '', 'finalized': False, 'good_expt': False, 'expt': 'primitives2', 'task_train_test': {'probe1_liketrain': 'train', 'probe1_nostrokeconstraint': 'train', 'probe2_liketrain': 'train', 'probe2_nostrokeconstraint': 'train', 'probe3_hdpos': 'test', 'probe1': 'train', 'probe2': 'train', 'probe3': 'test', 'probe4': 'test', 'train': 'train'}}

Getting these dates:
['210606', '210607', '210608']
Pancho 210606 primitives2 0
- No h5 file for Pancho, 210606, primitives2, 0 - returning None!
Pancho 210606 primitives2 1
- No h5 file for Pancho, 210606, primitives2, 1 - returning None!
Pancho 210606 primitives2 2
- No h5 file for Pancho, 210606, primitives2, 2 - returning None!
Pancho 210606 primit

#### DATASET FURTHER EXTRACTION (E..G, PARSES)

In [ ]:
from pythonlib.dataset.dataset import Dataset

for animal in ["Pancho", "Red"]:
    for expt in ["plandir2"]:
        
        #### LOAD
        D = Dataset([])
        D.load_dataset_helper(animal, expt)

        # PARSES
        D.planner_extract_save_all_parses() # parses
        

        # DEBUGGING
#         Dsub = D.copy()
#         Dsub.Dat = Dsub.Dat[:10]

#         Dsub.planner_score_beh_against_all_parses() # beh-task scores
#         Dsub.planner_score_parses() # parse scores

        D.planner_score_beh_against_all_parses() # beh-task scores
        D.planner_score_parses() # parse scores

## SCRATCH

In [ ]:
P.fd(0)["params"]

In [ ]:
expt

In [ ]:
sketchpad_edges_mk = fd["TrialRecord"]["User"]["BlockParams"]["1"]["sketchpad"]["edges_monkey"]
pos = convertCoords(fd, sketchpad_edges_mk.T, "monkeynorm2centeredmonkeypix")
sketchpad_edges_pixcentered = pos
sketchpad_edges_pixcentered

In [ ]:
print(SKIPS)

In [ ]:
P.generateDataset()

In [ ]:
from pythonlib.tools.pytools import get_size

get_size(P)

In [ ]:
print(get_size(P.Dataset)/1000)
print(get_size(P.Probedat)/1000)

In [ ]:
SF = P.Dataset.flattenToStrokdat()

In [ ]:
SF

In [ ]:
from analysis.strok import *
# == Filter
params = {
    "align_to_onset":True,
    "min_stroke_length_percentile":2,
    "min_stroke_length":50,
    "max_stroke_length_percentile":99.5,
}
SF = preprocessStroks(SF, params)



In [ ]:
rescale_strokes_ver = "stretch_to_1"
distancever = "euclidian_diffs"
npts_space = 50
Nbasis = 5

# 1) Get distance matrix, entire dataset, with random instances chosen for basis
# preprocess stroklist
stroklist = list(SF["strok"].values)

# rescale
if rescale_strokes_ver=="stretch_to_1":
    stroklist = [rescaleStrokes([s])[0] for s in stroklist]
else:
    print("keeping strokes scale unchaged")

# interpolate
if distancever in ["euclidian", "euclidian_diffs"]:
    # then need to be same length
    stroklist = strokesInterpolate2(stroklist, N=["npts", npts_space], base="space")

idxs_stroklist_dat = list(range(len(stroklist)))
idxs_stroklist_basis = random.sample(range(len(stroklist)), Nbasis)
similarity_matrix = distMatrixStrok(idxs_stroklist_dat, idxs_stroklist_basis, stroklist=stroklist,
                   normalize_rows=False, normalize_cols_range01=True, distancever=distancever)


In [ ]:
def F(x):
    indprobe = x["index_probedat"]
    return x["Probedat"].pandas().iloc[indprobe].to_dict()

from pythonlib.tools.pandastools import applyFunctionToAllRows

SF = applyFunctionToAllRows(SF, F)


### Test things about DAT

In [ ]:
fd = loadSingleDataQuick("Red", "210324", "figures9", 1)

In [ ]:
fd["TrialRecord"]["User"]["AdapterParams"]["50"]["bb"]["sequence"]

In [ ]:
DAT

In [ ]:
assert len(DAT)==len(P.pandas())
assert np.all(DAT.index == P.pandas().index)
assert (DAT["unique_task_name"] == P.pandas()["unique_task_name"]).all()
pd.merge(DAT, P.pandas(), how="outer")


In [ ]:
np.dtype('float64').itemsize
np.dtype('float16').itemsize

In [ ]:
# Does it help to convert data type from float64 (double) to float32?
# float 16 is a bit too imprecise.
# float32 doesnt make much of a difference (try code below)
# Therefore, dont make any change.

from pythonlib.tools.pytools import get_size

TYPE = "float16"
print(DAT["origin"][0].astype(TYPE))
print(get_size(DAT["origin"][0]))
get_size(DAT["origin"][0].astype(TYPE))

In [ ]:
DAT[:5]

### CONFIRMING that two datasets - DAT and DATloaded - are idencial. in places where they arent,  confirm that is different unique names due to diff hash method, or minor thing like in once case did not clean up short strokes

In [ ]:


import pickle
fname = "/data2/analyses/database/expts/Pancho-lines5-formodeling-210309_111226/dat.pkl"
with open(fname, "rb") as f:
    DATloaded = pickle.load(f)

i = 0

inds = DAT["unique_task_name"].values==DATloaded["unique_task_name"].values
print(DAT[inds][i])
print(DATloaded[inds][i])


# [np.isclose(d1, d2) for d1, d2 in zip(DAT["strokes_beh"].values, DATloaded["strokes_beh"].values)]
collect = 0
collect_names = []
for d1, d2 in zip(DAT["unique_task_name"].values, DATloaded["unique_task_name"].values):
    if d1!=d2:
        collect_names.append([d1, d2])
#         print(d1)
#         print(d2)
#         asdfsdf
    if len(d1) != len(d2):
        collect+=1
        if random.random()>0.8:
            print(len(d1), len(d2))
#             print(d1, d2)

            fig, ax = plt.subplots()
            plotDatStrokes(d1, ax)
            fig, ax = plt.subplots()
            plotDatStrokes(d2, ax)
#         assert False
#     try:
#         compareTasks(d1, d2, input_strokes_directly=True)
#     except:
#         print(d1)
#         print(d2)
#         safsadf
    
print("this many cases with strokes not matching number")
print(collect)

for c in collect_names:
    tmp =[]
    for cc1, cc2 in zip(c[0], c[1]):
        tmp.append(cc1==cc2)
    print(np.array(tmp).astype("float"))
#     print(c)

In [ ]:
# ## SCRATCH

# import seaborn as sns
# sns.catplot(data=P.pandas(), x='date')

# idx = 1001

# xyt = getTrialsPeanutPos(*P.fd_trial(idx))
# strokes_beh = getTrialsStrokesByPeanuts(*P.fd_trial(idx))
# strokes_task = getTrialsTaskAsStrokes(*P.fd_trial(idx))

# fig, ax = plt.subplots()
# plotDatStrokes(getTrialsTaskAsStrokes(*P.fd_trial(idx)), ax=ax)
# plotDatStrokes(strokes_beh, ax=ax)

## Throw out trials that are total garbage
## determine threshold for hausdorff.
# thresh_hausdorff = -0.97210514 # 2.5th percentile

### BELOW, looking at data to decide good extraction params. 

## Finding good hausodrff threshold for filtering data

In [ ]:
sum(np.isnan(Pp["frac_touched"].values))
print(np.percentile(Pp["hausdorff"].values, [1, 2.5, 5, 10]))
print(np.percentile(Pp["frac_touched"].values, [1, 5, 10]))


In [ ]:
sns.pairplot(data=Pp, vars=["hausdorff", "frac_touched"], height=8)

In [ ]:
# Plot hist of hausdorffs
plt.figure()
plt.hist(Pp["hausdorff"].values, 100)
plt.figure()
plt.hist(Pp["frac_touched"].values, 100)

# Pull out trials with bad hausdorff
# idxlist = np.where((Pp["hausdorff"].values<-0.8) & (Pp["hausdorff"].values>-1))
# idxlist = np.where((Pp["hausdorff"].values<-0.81788333) & (Pp["frac_touched"].values<0.8))
idxlist = np.where((Pp["hausdorff"].values<-1.49))
print(idxlist)

In [ ]:
# Plto those trials
plotTrialSingleOverview(*P.fd_trial(7121));

## BELOW, validating sketchpad, finding tyrials that extend beyond, and proprotions of trials doing so.
### Overall validates that the sketchpad (plus like 20 pix) is good
### Some extend beyond becuase subjects keep dragging, and is counted as part of stroke due to how peanutpus is extract.

In [ ]:
## Find, in practice, edges of behavior
def F(x):
    out = np.array([
        np.min([np.min(xx[:,0]) for xx in x["strokes_beh"]]),
        np.max([np.max(xx[:,0]) for xx in x["strokes_beh"]]),
        np.min([np.min(xx[:,1]) for xx in x["strokes_beh"]]),
        np.max([np.max(xx[:,1]) for xx in x["strokes_beh"]])
    ])
    return out
    
from pythonlib.tools.pandastools import applyFunctionToAllRows

# For each trial, get its min, max, for x and y
DAT2 = applyFunctionToAllRows(DAT, F)



# ==== PLOT
beh_edges = np.stack(DAT2["newcol"].values)

for i in range(beh_edges.shape[1]):
    print([np.min(beh_edges[:, i]), np.max(beh_edges[:, i])])


## PLOT DISTRIBITUSIONS OF EDGES (BEH) ACROSS ALL TASKS.
plt.figure()
plt.hist(beh_edges[:,0], 100)
plt.figure()
plt.hist(beh_edges[:,1], 100)
plt.figure()
plt.hist(beh_edges[:,2], 100)
plt.figure()
plt.hist(beh_edges[:,3], 100)

In [ ]:
sketchpad_edges_pixcentered.reshape(-1)

In [ ]:
# Find those trials exceeding sketchpad boundaries...
idx = np.where(beh_edges[:, 2]<-307.2)
print(idx)
print(beh_edges[idx, 2])
print("sketchpad edges:")
print(sketchpad_edges_pixcentered)

In [ ]:
# Get proportion of trials that dont fit into the sketchpad

edges = sketchpad_edges_pixcentered.reshape(-1)
edges = [edges[0]-10, edges[1]-10, edges[2]+10, edges[3]+10]
out = []
for i in range(beh_edges.shape[0]):
    b = beh_edges[i,:]
    tmp = [
        b[0]<edges[0], 
        b[1]>edges[2], 
        b[2]<edges[1], 
        b[3]>edges[3], 
    ]
    out.append(any(tmp))

print(f"{np.sum(out)} / {len(out)} ({np.sum(out)/len(out):.3f}) not fitting fully into sketchpad")


In [ ]:
## To example a particular trial (by plotting, checking video, etc)
ind = 4516

# find details of this trial
P.pandas().iloc[ind]

# Plot behavior on this trial.
plotTrialSingleOverview(*P.fd_trial(ind));